# Testing file

In [1]:
# from argoverse.data_loading.argoverse_tracking_loader import argoverse_tracking_loader

import argoverse
from argoverse.data_loading.argoverse_tracking_loader import ArgoverseTrackingLoader
import os
from shutil import copyfile
from argoverse.utils import calibration
import json
import numpy as np
from argoverse.utils.calibration import CameraConfig
from argoverse.utils.cv2_plotting_utils import draw_clipped_line_segment
from argoverse.utils.se3 import SE3
from argoverse.utils.transform import quat2rotmat
import math
import os
from typing import Union
import numpy as np
import pyntcloud
import progressbar
from time import sleep

In [2]:
root_dir = 'C://users/cathx/repos/argoverse-api/'
data_dir = root_dir + 'train/'

In [3]:
argoverse_loader = ArgoverseTrackingLoader(data_dir)

In [6]:
cams = ['ring_front_center',
'ring_front_left',
'ring_front_right',
'ring_rear_left',
'ring_rear_right',
'ring_side_left',
'ring_side_right',
'stereo_front_left',
'stereo_front_right']

In [10]:
for log_id in argoverse_loader.log_list:
    argoverse_data = argoverse_loader.get(log_id)
    for cam in cams:
        calibration_data = calibration.load_calib(data_dir+log_id+'/vehicle_calibration_info.json')[cam]
#         print(calibration_data)

In [4]:
calibration_data = calibration.load_calib(data_dir+log_id+'/vehicle_calibration_info.json')
print(calibration_data)

NameError: name 'log_id' is not defined

In [9]:
max_d = 100

_PathLike = Union[str, 'os.PathLike[str]']

def load_ply(ply_fpath: _PathLike) -> np.ndarray:
    """
    Load a point cloud file from a filepath.
    Args:
        ply_fpath: Path to a PLY file
    Returns:
        arr: array of shape (N, 3)
    """
    data = pyntcloud.PyntCloud.from_file(os.fspath(ply_fpath))
    x = np.array(data.points.x)[:, np.newaxis]
    y = np.array(data.points.y)[:, np.newaxis]
    z = np.array(data.points.z)[:, np.newaxis]

    return np.concatenate((x, y, z), axis=1)

data_dir = root_dir + 'train/'
goal_dir = root_dir + 'train_test_kitti/'
if not os.path.exists(goal_dir):
    os.mkdir(goal_dir)
    os.mkdir(goal_dir+'velodyne')
    os.mkdir(goal_dir+'image_2')
    os.mkdir(goal_dir+'image_3')
    os.mkdir(goal_dir+'calib')
    os.mkdir(goal_dir+'label_2')
    os.mkdir(goal_dir+'velodyne_reduced')


argoverse_loader = ArgoverseTrackingLoader(data_dir)
print('\nTotal number of logs:',len(argoverse_loader))
argoverse_loader.print_all()
print('\n')



Total number of logs: 20
#images: 83515
#lidar: 3794




In [10]:
# total_n = 0
# for q in argoverse_loader.log_list:
#     path, dirs, files = next(os.walk(data_dir + q + '/lidar'))
#     total_n = total_n+len(files)

# print(total_n)
# these are the number of lidar scans, we go ahead and multiply it by 7. For the sake of 7 cameras?
# 7 ring cameras and 2 stereo cameras 

In [11]:
cams = ['stereo_front_left', 'stereo_front_right']
i = 0
for log_id in argoverse_loader.log_list:
    argoverse_data = argoverse_loader.get(log_id)
    for cam in cams:
        # recreate the calibration file content
        # load calibration based on the camera from the cameras list
        # because of the for loop, we go camera by camera
        calibration_data = calibration.load_calib(data_dir+log_id+'/vehicle_calibration_info.json')[cam]
        # create new variable
        L3='P2: '
        for j in calibration_data.K.reshape(1,12)[0]:
            L3 = L3 + str(j) + ' '
        L3 = L3[:-1]

        L6 = 'Tr_velo_to_cam: '
        for k in calibration_data.extrinsic.reshape(1,16)[0][0:12]:
            L6 = L6+str(k)+' '
        L6 = L6[:-1]

        L1 ='P0: 0 0 0 0 0 0 0 0 0 0 0 0'
        L2 ='P1: 0 0 0 0 0 0 0 0 0 0 0 0'
        L4 ='P3: 0 0 0 0 0 0 0 0 0 0 0 0'
        L5 ='R0_rect: 1 0 0 0 1 0 0 0 1'
        L7 ='Tr_imu_to_velo: 0 0 0 0 0 0 0 0 0 0 0 0'

        file_content = """{}
{}
{}
{}
{}
{}
{}
        """.format(L1, L2, L3, L4, L5, L6, L7)
        l=0

        for timestamp in argoverse_data.lidar_timestamp_list:
            lidar_file_path = data_dir + log_id + '/lidar/PC_' + str(timestamp) +'.ply'
            target_lidar_file_path = goal_dir + 'velodyne/' + str(i).zfill(6) + '.bin'
            lidar_data = load_ply(lidar_file_path)
            lidar_data_augmented = np.concatenate((lidar_data, np.zeros([lidar_data.shape[0],1])), axis=1)
            lidar_data_augmented = lidar_data_augmented.astype('float32')
            lidar_data_augmented.tofile(target_lidar_file_path)

            cam_file_path = argoverse_data.image_list_sync[cam][l]
            target_cam_file_path = goal_dir + 'image_2/' + str(i).zfill(6) + '.png'
            copyfile(cam_file_path, target_cam_file_path)

            # cam_file_path = argoverse_data.image_list_sync[cam][l]
            # target_cam_file_path = goal_dir + 'image_3/' + str(i).zfill(6) + '.png'
            # copyfile(cam_file_path, target_cam_file_path)

            file=open(goal_dir + 'calib/' + str(i).zfill(6) + '.txt', 'w+')
            file.write(file_content)
            file.close()

            label_object_list = argoverse_data.get_label_object(l)
            file=open(goal_dir + 'label_2/' + str(i).zfill(6) + '.txt', 'w+')
            l +=1

            for detected_object in label_object_list:
                classes = detected_object.label_class
                occulusion = round(detected_object.occlusion/25)
                height = detected_object.height
                length = detected_object.length
                width = detected_object.width
                truncated = 0

                center = detected_object.translation # in ego frame

                corners_ego_frame = detected_object.as_3d_bbox()
                corners_cam_frame = calibration_data.project_ego_to_cam(corners_ego_frame)
                image_corners = calibration_data.project_ego_to_cam(corners_ego_frame)
                image_bbox = [min(image_corners[:, 0]), min(image_corners[:,1]),max(image_corners[:,0]),max(image_corners[:,1])]
                image_bbox = [round(x) for x in image_bbox]

                center_cam_frame = calibration_data.project_ego_to_cam(np.array([center]))

                if 0 <center_cam_frame[0][2]<max_d and 0<image_bbox[0]<1920 and 0<image_bbox[1]<1200 and 0<image_bbox[2]<1920 and 0<image_bbox[3]<1200:

                    # the center frames we need for KITTI
                    p1 = corners_cam_frame[1]
                    p5 = corners_cam_frame[5]
                    dz = p1[2] - p5[2]
                    dx = p1[0] - p5[0]

                    angle = math.atan2(dz, dx)
                    beta = math.atan2(center_cam_frame[0][2],center_cam_frame[0][0])
                    alpha = angle + beta - math.pi/2
                    line = classes+ ' {} {} {} {} {} {} {} {} {} {} {} {} {} {} \n'.format(round(truncated, 2), occulusion, round(alpha,2), round(image_bbox[0],2), round(image_bbox[1], 2),\
                    round(image_bbox[2], 2), round(image_bbox[3], 2), round(height,2), round(width,2), round(length,2),round(center_cam_frame[0][0],2),
                    round(center_cam_frame[0][1],2), round(center_cam_frame[0][2],2),round(angle,2))
                    file.write(line)
            file.close()
            i += 1

IndexError: list index out of range

In [47]:
_PathLike = Union[str, 'os.PathLike[str]']

def load_ply(ply_fpath: _PathLike) -> np.ndarray:
    """
    Load a point cloud file from a filepath.
    Args:
        ply_fpath: Path to a PLY file
    Returns:
        arr: array of shape (N, 3)
    """
    data = pyntcloud.PyntCloud.from_file(os.fspath(ply_fpath))
    x = np.array(data.points.x)[:, np.newaxis]
    y = np.array(data.points.y)[:, np.newaxis]
    z = np.array(data.points.z)[:, np.newaxis]

    return np.concatenate((x, y, z), axis=1)

lidar_file_path = root_dir + '/train/3d20ae25-5b29-320d-8bae-f03e9dc177b9/lidar/PC_315975023020283000.ply'
lidar_data = load_ply(lidar_file_path)
print(lidar_data)

    

[[  1.1149248   -2.146084     0.58089316]
 [  1.0111252  -11.640176     1.386621  ]
 [ -0.13876712 -11.654566     1.2477934 ]
 ...
 [ -0.033566     5.931162    -0.17467225]
 [ -0.4496715    6.1416583    0.30140412]
 [ -1.3329737    9.644014     1.7516748 ]]


In [33]:
L3 = L3[:-1]
print(L3)

P2: 3666.737987697073 0.0 1230.5215533709008 0.0 0.0 3666.737987697073 1059.5814244164 0.0 0.0 0.0 1.0 0.


In [6]:
paths = os.listdir('./train/')

In [7]:
for i in paths:
    complete = os.path.join('./train/' + i)
    print(complete)


./train/043aeba7-14e5-3cde-8a5c-639389b6d3a6
./train/10b3a1d8-e56c-38be-aaf7-ef2f862a5c4e
./train/10b8dee6-778f-33e4-a946-d842d2d9c3d7
./train/11953248-1195-1195-1195-511954366464
./train/230970eb-dc2e-3133-b252-ff3c6f5d4284
./train/25952736-2595-2595-2595-225953853440
./train/26d141ec-f952-3908-b4cc-ae359377424e
./train/273c1883-673a-36bf-b124-88311b1a80be
./train/3d20ae25-5b29-320d-8bae-f03e9dc177b9
./train/64c12551-adb9-36e3-a0c1-e43a0e9f3845
./train/6f153f9c-edc5-389f-ac6f-40705c30d97e
./train/84c35ea7-1a99-3a0c-a3ea-c5915d68acbc
./train/88538208-8853-8853-8853-388539396096
./train/8a15674a-ae5c-38e2-bc4b-f4156d384072
./train/a073e840-6319-3f0b-843e-f6dccdcc7b77
./train/b3def699-884b-3c9e-87e1-1ab76c618e0b
./train/c6911883-1843-3727-8eaa-41dc8cda8993
./train/dcdcd8b3-0ba1-3218-b2ea-7bb965aad3f0
./train/e17eed4f-3ffd-3532-ab89-41a3f24cf226
./train/f3fb839e-0aa2-342b-81c3-312b80be44f9


In [3]:
# main_dir = 'C://users/cathx/repos/argoverse-api/'
root_dir = 'C://Users/cathx/repos/argoverse-api/tracking_train1_v1.1.tar/argoverse-tracking/train1/'
paths = os.listdir(root_dir)
# paths = os.listdir(main_dir + './train/')
# for i in paths:
#     complete = os.path.join(main_dir + i)
#     print(complete)
# # print(paths)

In [5]:
for i in paths:
    complete = os.path.join(root_dir+ i)
#     print(complete)

In [49]:
for file in os.listdir(complete):
    for root, paths, filenames in os.walk(file + 'stere_front_left', topdown=False):
        for count, filename in enumerate(filenames):
             os.rename('stereo_front_left'+ str(count) + '.jpg')

In [11]:
for file in os.listdir(complete):
for root, dirs, files in os.walk(os.listdir(complete), topdown=False):
    for name in files:
        print(os.path.join(root, name))
    for name in dirs:
        print(os.path.join(root, name))

In [2]:
# for root, dirs, files in os.walk('C://users/cathx/repos/argoverse-api/train/c6911883-1843-3727-8eaa-41dc8cda8993/stereo_front_left/'):
# #     for name in files:
# #         print(os.path.join(root, name))
#     count = 0
#     for name in files:
#         new_name = 'stereo_front_left' + str(count) + '.jpg'
#         os.rename(name, 'stereo_front_left_'+str(count)+'.jpg')
#         count +=1

In [1]:
import os
for i in os.listdir(root_dir):
    path = (main_dir + i + '/stereo_front_right/')

    for count, filename in enumerate(os.listdir(main_dir + '/train/' + i + '/stereo_front_right')):
        new_name = 'stereo_front_right_' + "{:06}.jpg".format(str(count))
        source = path + filename
        new_name = path + new_name
        
        os.rename(source, new_name)

NameError: name 'root_dir' is not defined

In [14]:
root_dir = 'C://Users/cathx/repos/argoverse-api/tracking_train1_v1.1.tar/argoverse-tracking/train/'


In [8]:
import os
for i in os.listdir(root_dir):
    path = (root_dir + i + '/stereo_front_right/')

    for count, filename in enumerate(os.listdir(root_dir + i + '/stereo_front_right')):
        new_name = "{:06}.jpg".format((count))
        source = path + filename
        new_name = path + new_name
        
        os.rename(source, new_name)

In [32]:
for i in paths:
    complete = os.path.join(root_dir + 'train/' + i)
    print(complete)

C://Users/cathx/repos/argoverse-api/tracking_train1_v1.1.tar/argoverse-tracking/train1/train/043aeba7-14e5-3cde-8a5c-639389b6d3a6
C://Users/cathx/repos/argoverse-api/tracking_train1_v1.1.tar/argoverse-tracking/train1/train/10b3a1d8-e56c-38be-aaf7-ef2f862a5c4e
C://Users/cathx/repos/argoverse-api/tracking_train1_v1.1.tar/argoverse-tracking/train1/train/10b8dee6-778f-33e4-a946-d842d2d9c3d7
C://Users/cathx/repos/argoverse-api/tracking_train1_v1.1.tar/argoverse-tracking/train1/train/11953248-1195-1195-1195-511954366464
C://Users/cathx/repos/argoverse-api/tracking_train1_v1.1.tar/argoverse-tracking/train1/train/230970eb-dc2e-3133-b252-ff3c6f5d4284
C://Users/cathx/repos/argoverse-api/tracking_train1_v1.1.tar/argoverse-tracking/train1/train/25952736-2595-2595-2595-225953853440
C://Users/cathx/repos/argoverse-api/tracking_train1_v1.1.tar/argoverse-tracking/train1/train/26d141ec-f952-3908-b4cc-ae359377424e
C://Users/cathx/repos/argoverse-api/tracking_train1_v1.1.tar/argoverse-tracking/train1/tra

In [26]:
paths = os.listdir(os.path.join(root_dir + i))
print(paths)

['city_info.json', 'lidar', 'per_sweep_annotations_amodal', 'poses', 'ring_front_center', 'ring_front_left', 'ring_front_right', 'ring_rear_left', 'ring_rear_right', 'ring_side_left', 'ring_side_right', 'stereo_front_left', 'stereo_front_right', 'track_labels_amodal', 'vehicle_calibration_info.json']


In [ ]:
paths = [i for i in os.listdir(root_dir)]

In [13]:
# def videos(root_dir):
#     for i in os.listdir(root_dir):
#         path = (root_dir + i + '/stereo_front_right/')
#         print(os.listdir(path))
#         images = [img for img in os.listdir(path) if img.endswith(".jpg")]

#         frame = cv2.imread(os.path.join(path, images[0]))
#         height, width, layers = frame.shape

#         video_name = 'left_cam.avi'
#         fps = 5
#         count = 0
#         video_name = f('test_vid_{count}'+'avi'
#         video = cv2.VideoWriter(os.path.join(root_dir+i+ video_name), 0, fps, (width, height))

#         for image in images:
#             video.write(cv2.imread(os.path.join(image_folder, image)))



In [ ]:
def videos_write(root_dir):
    
    paths = os.listdir(os.path.join(root_dir + i)
    for i in paths:
        complete = os.path.join(main_dir + 'train/' + i)
        

In [ ]:
for i in os.listdir(main_dir + 'train/'):
    path = (main_dir + '/train/' + i + '/stereo_front_right/')

    for count, filename in enumerate(os.listdir(main_dir + '/train/' + i + '/stereo_front_right')):
        new_name = 'stereo_front_right_' + str(count) + ".jpg"
        source = path + filename
        new_name = path + new_name
        
        os.rename(source, new_name)

In [ ]:
import os
for i in os.listdir(root_dir):
    path = (root_dir + i + '/stereo_front_right/')

    for count, filename in enumerate(os.listdir(root_dir + i + '/stereo_front_right')):
        new_name = "{:06}.jpg".format((count))
        source = path + filename
        new_name = path + new_name
        
        os.rename(source, new_name)

In [9]:
from PIL import Image
import glob 

OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'C://Users/cathx/repos/argoverse-api/tracking_train1_v1.1.tar/argoverse-tracking/train/f3fb839e-0aa2-342b-81c3-312b80be44f9/stereo_front_right/*.jpg'

# Loading Calibration data

In [28]:
print('\nLoading files...')

import argoverse
from argoverse.data_loading.argoverse_tracking_loader import ArgoverseTrackingLoader
import os
from shutil import copyfile
from argoverse.utils import calibration
import json
import numpy as np
from argoverse.utils.calibration import CameraConfig
from argoverse.utils.cv2_plotting_utils import draw_clipped_line_segment
from argoverse.utils.se3 import SE3
from argoverse.utils.transform import quat2rotmat
import math
import os
from typing import Union
import numpy as np
import pyntcloud
import progressbar
from time import sleep


Loading files...


In [25]:
root_dir = 'C://users/cathx/repos/argoverse-api/'
data_dir = root_dir + '/train/'


In [29]:
_PathLike = Union[str, "os.PathLike[str]"]

In [30]:
def load_ply(ply_fpath: _PathLike) -> np.ndarray:
    """Load a point cloud file from a filepath.
    Args:
        ply_fpath: Path to a PLY file
    Returns:
        arr: Array of shape (N, 3)
    """

    data = pyntcloud.PyntCloud.from_file(os.fspath(ply_fpath))
    x = np.array(data.points.x)[:, np.newaxis]
    y = np.array(data.points.y)[:, np.newaxis]
    z = np.array(data.points.z)[:, np.newaxis]

    return np.concatenate((x, y, z), axis=1)